In [9]:
# !pip install apache-beam[gcp] pyarrow pandas google-cloud-storage google-cloud-bigquery

In [10]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import storage
from google.cloud import bigquery

# Define o projeto do Google Cloud
PROJECT_ID = "techchallenge-2"
BUCKET_NAME = "bovespa-dados-brutos"

print("Autenticação concluída! ✅")


Autenticação concluída! ✅


In [11]:
import os

# Nome do arquivo Parquet a testar (mude conforme necessário)
GCS_PARQUET_PATH = "bovespa_raw/data=20250305/Empresas_IBOV_20250305_20250305_005039.parquet"
LOCAL_PARQUET_PATH = "/content/test_data.parquet"

# Conectar ao GCS e baixar o arquivo Parquet
client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(GCS_PARQUET_PATH)
blob.download_to_filename(LOCAL_PARQUET_PATH)

print(f"Arquivo {GCS_PARQUET_PATH} baixado para {LOCAL_PARQUET_PATH} ✅")


Arquivo bovespa_raw/data=20250305/Empresas_IBOV_20250305_20250305_005039.parquet baixado para /content/test_data.parquet ✅


In [12]:
import pyarrow.parquet as pq
import pandas as pd

# Carregar o Parquet em um DataFrame
df = pq.read_table(LOCAL_PARQUET_PATH).to_pandas()

# Visualizar as 5 primeiras linhas do arquivo
df.head()


,NME_SETOR,COD_EMPRESA,NME_ACAO,DSC_TIPO,VAL_PARTICIPACAO_SETOR,VAL_PARTICIPACAO_ACUMULADA_SETOR,QTD_TEORICA,DT_REFERENCIA_CARTEIRA,DT_EXTRACAO
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON ED NM,"3,014","3,014",1.243.177.587,20250305,2025-03-05 00:50:39
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,"2,580","2,827",734.631.801,20250305,2025-03-05 00:50:39
2,Bens Indls / Mat Transporte,POMO4,MARCOPOLO,PN ED N2,"0,247","2,827",666.378.439,20250305,2025-03-05 00:50:39
3,Bens Indls/Transporte,AZUL4,AZUL,PN ES N2,"0,064","2,263",326.872.005,20250305,2025-03-05 00:50:39
4,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,"0,585","2,263",991.920.937,20250305,2025-03-05 00:50:39


In [13]:
import re

def limpar_numero(valor):
    """Remove separadores de milhar, troca vírgula decimal por ponto e converte para float."""
    if pd.isna(valor) or valor is None or valor == "":
        return 0.0  # Retorna 0.0 para valores nulos ou vazios

    if isinstance(valor, (int, float)):
        return float(valor)  # Se já for numérico, retorna diretamente

    if isinstance(valor, str):
        valor = valor.strip()  # Remove espaços extras
        valor = re.sub(r"[^\d,.-]", "", valor)  # Mantém apenas números, pontos e vírgulas
        valor = valor.replace(".", "").replace(",", ".")  # Remove separadores de milhar e ajusta decimal

        try:
            return float(valor)  # Converte para float
        except ValueError:
            print(f"⚠️ Erro ao converter para float: '{valor}'")  # Debugging opcional
            return 0.0  # Retorna 0.0 se houver erro na conversão

    return 0.0  # Valor padrão para casos inesperados

# Testar com valores do DataFrame
df["VAL_PARTICIPACAO_SETOR_LIMPO"] = df["VAL_PARTICIPACAO_SETOR"].apply(limpar_numero)

# Mostrar os dados limpos
df[["VAL_PARTICIPACAO_SETOR", "VAL_PARTICIPACAO_SETOR_LIMPO"]].head()


,VAL_PARTICIPACAO_SETOR,VAL_PARTICIPACAO_SETOR_LIMPO
0,"3,014",3.014
1,"2,580",2.580
2,"0,247",0.247
3,"0,064",0.064
4,"0,585",0.585


In [14]:
def transformar_dados(element):
    """Converte os dados para o formato correto para o BigQuery"""
    return {
        "dt_referencia_carteira": int(element.get("DT_REFERENCIA_CARTEIRA", 0)),
        "dt_extracao": element.get("DT_EXTRACAO", ""),
        "cod_empresa": element.get("COD_EMPRESA", ""),
        "nme_acao": element.get("NME_ACAO", ""),
        "dsc_tipo": element.get("DSC_TIPO", ""),
        "val_participacao_setor": limpar_numero(element.get("VAL_PARTICIPACAO_SETOR", "0")),
        "val_participacao_acumulada_setor": limpar_numero(element.get("VAL_PARTICIPACAO_ACUMULADA_SETOR", "0")),
        "qtd_teorica": int(limpar_numero(element.get("QTD_TEORICA", "0"))),
        "nme_setor": element.get("NME_SETOR", ""),
    }

# Aplicar a transformação nos primeiros 5 registros
df_transformado = df.apply(transformar_dados, axis=1)

# Mostrar os dados transformados
df_transformado


,0
0,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
1,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
2,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
3,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
4,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
...,...
82,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
83,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
84,"{'dt_referencia_carteira': 20250305, 'dt_extra..."
85,"{'dt_referencia_carteira': 20250305, 'dt_extra..."


In [15]:
# Definir as variáveis antes de usá-las
PROJECT_ID = "techchallenge-2"  # ID do seu projeto no Google Cloud
DATASET_ID = "bovespa_dataset"  # Nome do dataset no BigQuery
TABLE_NAME = "empresas_ibov"  # Nome da tabela

# Criar a referência da tabela corretamente
TABLE_REF = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}"

print("TABLE_REF definido como:", TABLE_REF)


TABLE_REF definido como: techchallenge-2.bovespa_dataset.empresas_ibov


In [16]:
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Conectar ao BigQuery
client_bq = bigquery.Client(project=PROJECT_ID)

# Excluir a tabela existente, se necessário
client_bq.delete_table(TABLE_REF, not_found_ok=True)
print(f"🚨 Tabela {TABLE_REF} excluída!")

# Criar DataFrame (caso ainda não tenha sido definido)
df_bq = pd.DataFrame(df_transformado.tolist())  # Certifique-se de que df_transformado existe

# Criar a nova tabela com o esquema atualizado
to_gbq(df_bq, TABLE_REF, project_id=PROJECT_ID, if_exists="replace")

print("✅ Nova tabela criada com sucesso no BigQuery!")


🚨 Tabela techchallenge-2.bovespa_dataset.empresas_ibov excluída!


100%|██████████| 1/1 [00:00<00:00, 1832.37it/s]

✅ Nova tabela criada com sucesso no BigQuery!
